In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

In [14]:
file_path = "C:/Naman/Programming/AI Craft/exoTrain.csv"
df = pd.read_csv(file_path)

In [15]:
# Extract features and labels
X = df.iloc[:, 1:].values  # Flux values
y = df.iloc[:, 0].values   # Labels

In [16]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [17]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [18]:
# Reshape input for LSTM (samples, time steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [19]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [20]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [21]:
# Create DataLoaders
batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        x = self.dropout(lstm_out[:, -1, :])
        x = self.fc(x)
        return torch.sigmoid(x)

In [23]:
# Model parameters
input_size = 1
hidden_size = 64
num_layers = 2
output_size = 1

In [24]:
# Initialize model
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train model
epochs = 50
for epoch in range(epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Evaluate model
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X).squeeze()
        predicted = (outputs > 0.5).float()
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

print(f"Test Accuracy: {correct / total:.4f}")

Epoch [1/50], Loss: 0.0059
Epoch [2/50], Loss: 0.0101
Epoch [3/50], Loss: 0.0054
Epoch [4/50], Loss: 0.0104
Epoch [5/50], Loss: 0.0066
Epoch [6/50], Loss: 0.0046
Epoch [7/50], Loss: 0.0079
Epoch [8/50], Loss: 0.0076
Epoch [9/50], Loss: 0.0055
Epoch [10/50], Loss: 0.0049
Epoch [11/50], Loss: 0.0079
Epoch [12/50], Loss: 0.0088
Epoch [13/50], Loss: 0.9400
Epoch [14/50], Loss: 0.0077
Epoch [15/50], Loss: 0.0081
Epoch [16/50], Loss: 0.0046
Epoch [17/50], Loss: 0.0102
Epoch [18/50], Loss: 0.0099
Epoch [19/50], Loss: 0.0058
Epoch [20/50], Loss: 0.0079
Epoch [21/50], Loss: 0.0080
Epoch [22/50], Loss: 0.0065
Epoch [23/50], Loss: 0.0080
Epoch [24/50], Loss: 0.0094
Epoch [25/50], Loss: 0.0062
Epoch [26/50], Loss: 0.0062
Epoch [27/50], Loss: 0.0061
Epoch [28/50], Loss: 0.0058
Epoch [29/50], Loss: 0.0054
Epoch [30/50], Loss: 0.0069
Epoch [31/50], Loss: 0.0056
Epoch [32/50], Loss: 0.0071
Epoch [33/50], Loss: 0.0058
Epoch [34/50], Loss: 0.0052
Epoch [35/50], Loss: 0.0075
Epoch [36/50], Loss: 0.0052
E